# Наибольший общий делитель

In [4]:
import time
import numpy as np
np.set_printoptions(suppress=True)

###  НОД через вычитание

In [5]:
def timing(function):
    import time   
    
    def wrapper(a, b):
        start = time.time()
        print(function(a, b))
        stop = time.time()
        print('{:f} секунд'.format(stop - start))
    return wrapper

In [6]:
@timing
def nod(a, b):
    if a == b:
        return a
    while a != b:
        if a > b:
            a = a - b
        else:
            b = b - a
    return a

In [7]:
nod(125, 15)

5
0.000139 секунд


In [8]:
def timing_common(function):
    start = time.time()
    print(function)
    stop = time.time()
    print('{:f}'.format(stop - start))

### НОД через остаток от деления

In [9]:
@timing
def nod_mod(a, b):
    if a == 0:
        return b
    if b == 0:
        return a
    while((a != 0) and (b != 0)):
        if a > b:
            a = a % b
        else:
            b = b % a
    return max([a, b])

In [10]:
nod_mod(2334567890, 12)

2
0.000138 секунд


### НОД рекурсия

In [11]:
def gcd(a, b):
    if (b == 0): 
        return a
    return gcd(b, a % b)

In [13]:
timing_common(gcd(15, 125))

5
0.000138


In [14]:
timing_common(gcd(12334567890, 12))

6
0.000435


### сдвиг с рекурсией

In [16]:
def bin_gcd(a, b):
    if a == b:
        return a
    if a == 0:
        return b
    if b == 0:
        return a
    
    if(~a & 1): # a - четн
        if(b & 1): # b - нечет
            return bin_gcd(a >> 1, b)
        else: # a и b  - четн
            return bin_gcd(a >> 1, b >> 1) << 1
        
    if(~b & 1):  # a  - нечет, b  - четн
        return bin_gcd(a, b >> 1)

    
    if(a > b):
        return bin_gcd((a - b) >> 1, b)
    else:
        return bin_gcd((b - a) >> 1, a)

In [17]:
timing_common(bin_gcd(12334567890, 12))

6
0.000055


### сдвиг с циклами

In [19]:
@timing
def bin_iter_gcd(a, b):
    shift = 0
    if a == 0:
        return b
    if b == 0:
        return a
    
    while((( a | b) & 1) == 0):
        shift += 1
        a >>= 1
        b >>= 1
    
    while((a & 1) == 0):
        b >>= 1
        
    while True:
        while((b & 1) == 0):
            b >>= 1
        if(a > b):
            a, b = b, a
        b -= a
        
        if(b == 0):
            break
    
    return a << shift
        

In [21]:
bin_iter_gcd(12334567890, 12)

6
0.000380 секунд


### таблица сравнения

In [23]:
a = 12334567890
b = 12

In [24]:
print('НОД вычитание')
nod(a,b)
print('----------')
print('НОД остаток от деления')
nod_mod(a, b)
print('----------')
print('НОД рекрсия')
timing_common(gcd(a, b))
print('----------')
print('НОД битовые операции с рекурсией')
timing_common(bin_gcd(a, b))
print('----------')
print('НОД битовые операции с циклом')
bin_iter_gcd(a, b)

НОД вычитание
6
50.945933 секунд
----------
НОД остаток от деления
6
0.000012 секунд
----------
НОД рекрсия
6
0.000012
----------
НОД битовые операции с рекурсией
6
0.000017
----------
НОД битовые операции с циклом
6
0.000019 секунд


# Степень

### итеративный

In [25]:
@timing
def power_iter(base, power):
    result = 1
    for _ in range(0, power):
        result *= base
    return result

In [26]:
power_iter(2, 0)

1
0.000583 секунд


### быстрое возведение в степень

In [27]:
@timing
def power_m(base, power):
    res = 1
    while power > 1:
        if power % 2 == 1:
            res *= base
        base *= base
        power //= 2
    if power > 0:
        res *= base
    return res

In [28]:
power_m(2, 0)

1
0.000045 секунд


In [29]:
pow(2,0)

1

In [30]:
power_m(1.000001, 1000000)

2.7182804691564275
0.000138 секунд


$e = \lim_{x\to\infty}(1+ 1/n)^n$

### степень двойки с домножением

In [31]:
@timing
def pow_2_half(base, power):
    a = base
    if power == 0:
        return 1
    temp = 2
    while (temp <= power):
        a *= a
        temp += temp
    if temp / 2 == power:
        return a
    else:
        for item in range(power - temp//2):
            a *= base
    return a
    

In [32]:
pow_2_half(1.000001, 1000000)

2.7182804691276115
0.014153 секунд


### таблица сравнения

In [33]:
print('Итеративный алгоритм')
power_iter(1.000001, 1000000)
print('----------')
print('степень двойки с домножением')
pow_2_half(1.000001, 1000000)
print('----------')
print('быстрое возведение в степень')
power_m(1.000001, 1000000)
print('----------')

Итеративный алгоритм
2.7182804690959363
0.031829 секунд
----------
степень двойки с домножением
2.7182804691276115
0.015445 секунд
----------
быстрое возведение в степень
2.7182804691564275
0.000035 секунд
----------


# Алгоритм нахождения всех простых чисел дл N

### сначала - определить, является ли число N простое

### полный перебор

In [34]:
def prime(p):
    d = 0
    for item in range(1, p + 1):
        if ( p % item == 0):
            d += 1
    return d == 2

In [35]:
prime(1997)

True

In [36]:
@timing
def count_prime(number, function):
    count = 0
    for item in range(2, number +1):
        if function(item):
            count +=1
    return count

    

In [37]:
count_prime(100000, prime)

9592
228.660486 секунд


### выбросим все четные (можно и не делеить на четные)

In [43]:

def prime_1(p):
    if (p == 2):
        return True
    if (~p&1):
        return False
    for item in range(3, p, 2):
#         print(item)
        if(p % item == 0):
            return False
    return True

In [44]:
count_prime(100000, prime_1)

9592
10.212165 секунд


### перебор до корня из p

In [45]:
import math
def prime_2(p):
    if p == 2:
        return True
    if (~p&1):
        return False   
    border = round(math.sqrt(p)) + 1
    for item in range(3, border, 2):
        if ( p % item == 0):
            return False
    return True

In [46]:
count_prime(100000, prime_2)

9592
0.077671 секунд


### сохранять простые числа и делить на них

In [47]:
import math
def count_prime_with_storage_prime(p):
    primes = []
    count = 0
    
    def prime_3(p):
        if p == 2:
            return True
        if (~p&1):
            return False       
        border = round(math.sqrt(len(primes)))
        for item in range(border):
            if ( p % primes[item] == 0):
                return False
        return True
    
    
    for item in range(2, 100000 +1):
        if prime_3(item):
            primes.append(item)
            count +=1
    return count
            


In [48]:
timing_common(count_prime_with_storage_prime(100000))

9592
0.000054


### решето с for

In [49]:
def eratosthenes(n):     
    separator = list(range(n + 1))
    separator[1] = 0
    for i in separator:
        if i > 1:
            for j in range(i * i, len(separator), i):
                separator[j] = 0
    answer = set(separator)
    answer.remove(0)
    return len(list(answer))

In [50]:
timing_common(eratosthenes(100000))

9592
0.000159


### Решето c while  и numpy

In [51]:
import numpy as np
def eratosphens_separator(n):
    arr = np.array(range(0, n+1))
    arr[1] = 0
    i= 2
    while i <= n:
        if arr[i] != 0:
            temp =  i * i
            while temp <= n:
                arr[temp] = 0
                temp += i
        i += 1
    
    prime_arr = set(arr)
    prime_arr.remove(0)
    return len(list(prime_arr))
    

In [52]:
timing_common(eratosphens_separator(100000))

9592
0.000116


### Решето за O(n)

In [53]:
def eratosphen_o_n(n):
    lp = [0] * (n + 1)
    pr = []
    
    for i in range(2, n + 1):
        if lp[i] == 0:
            lp[i] = i
            pr.append(i)
        p = 0
        while ((p < len(pr)) and (pr[p] <= lp[i]) and (i * pr[p] <= n )):
            lp[pr[p] * i] = pr[p]
#             print('pr[p] =', pr[p], ' * ', 'i= ', i, '->', pr[p])
            p += 1
#             print(lp)
#             print(pr)
    return len(pr)    

In [54]:
timing_common(eratosphen_o_n(100000))

9592
0.000140


### таблица

In [55]:
print('Полный перебор чисел')
count_prime(100000, prime)
print('----------')
print('Выброшены все четные')
count_prime(100000, prime_1)
print('----------')
print('Перебор дл корня из p')
count_prime(100000, prime_2)
print('----------')
print('Cохранять простые числа и делить на них')
timing_common(count_prime_with_storage_prime(100000))
print('----------')
print('Решето Эратосфена, вариант 1')
timing_common(eratosthenes(100000))
print('----------')
print('Решето Эратосфена, на numpy')
timing_common(eratosphens_separator(100000))
print('----------')
print('Решето Эратосфена, за O(n)')
timing_common(eratosphen_o_n(100000))
print('----------')

Полный перебор чисел
9592
223.962711 секунд
----------
Выброшены все четные
9592
10.454550 секунд
----------
Перебор дл корня из p
9592
0.074153 секунд
----------
Cохранять простые числа и делить на них
9592
0.000056
----------
Решето Эратосфена, вариант 1
9592
0.000052
----------
Решето Эратосфена, на numpy
9592
0.000039
----------
Решето Эратосфена, за O(n)
9592
0.000043
----------


# Число Фибоначчи

### через рекурсию

In [56]:
def f(n):
    if n <= 2:
        return 1
    else:
        return f(n-1) + f(n-2)

In [57]:
timing_common(f(40))

102334155
0.000054


In [58]:
start = time.time()
print(f(40))
stop = time.time()
print('{:f} секунд'.format(stop - start))

102334155
16.294320 секунд


### Динамическое программирование

In [59]:
def fb(n):
    if n < 2 and n > 0:
        return 1
    a = 1
    b = 1
    for i in range(2, n ):
        f = a + b
        a, b = b, f
    return f

In [60]:
timing_common(fb(50))

12586269025
0.000139


In [61]:
start = time.time()
print(fb(50))
stop = time.time()
print('{:f} секунд'.format(stop - start))

12586269025
0.000270 секунд


### рекуррентная формула через золотое сечение

$a = (1 + sqrt(5))/2$

$ Fn = ф^n/sqrt(5) + 1/2$

In [62]:
import math
def fib_gold(n):
    sq = math.sqrt(5)
    f = (1 + sq)/2
    return int(f**n/sq + 0.5)

In [63]:
timing_common(fib_gold(40))

102334155
0.000048


### через матрицу перехода

In [64]:
import numpy as np
def fib_matrix(n):
    matrix = np.array([[1, 1], [1, 0]])
    return np.linalg.matrix_power(matrix, n-1)[0][0]
    

In [65]:
timing_common(fib_matrix(30))

832040
0.000482


### Таблица

In [66]:
print('Число Фибоначчи с помощью рекурсии')
start = time.time()
print(f(40))
stop = time.time()
print('{:f} секунд'.format(stop - start))
print('----------')

print('Динамическое программирование')
timing_common(fb(40))
print('----------')
print('Золотое сечение')
timing_common(fib_gold(40))
print('----------')
print('Матричный способ')
timing_common(fib_matrix(40))
print('----------')

Число Фибоначчи с помощью рекурсии
102334155
15.380250 секунд
----------
Динамическое программирование
102334155
0.000011
----------
Золотое сечение
102334155
0.000010
----------
Матричный способ
102334155
0.000018
----------
